# Javascript: Image Edge Detection Filter
<p align="left">
<img src="./img/businesscard.jpg" width="500px" alt="Business Card" align="left" >
</p>
<br>

[www.rptecnologias.com](http://www.rptecnologias.com)
<br>
guitars.ai@rptecnologias.com


In [1]:
// Imports
var jpeg = require('jpeg-js');
var fs = require('fs');
var conv = require('./javascript/convolution');

In [2]:
// Load JPEG Image
var jpegData = fs.readFileSync('./img/guitar.jpg');
var rawImageData = jpeg.decode(jpegData, true); // return as Uint8Array

In [3]:
// Define a function to convert RGBA to an Object containing 2D Arrays R,G,B and Alpha
function RGBAtoRGBAObj2D(rgba_arr)
{
    //Initialization
    var r=[], g=[], b=[], alpha=[];
    var index=0, n=0;
    r[0]=[];
    g[0]=[];
    b[0]=[];
    alpha[0]=[];
    
    for(var p = 0; p < rgba_arr.data.length; p+=4) 
    {

    r[index][n] = rgba_arr.data[p];
    g[index][n] = rgba_arr.data[p+1];
    b[index][n] = rgba_arr.data[p+2];
    alpha[index][n] = rgba_arr.data[p+3];
    n++;
    if(n%rgba_arr.width==0 && n!=0)
        {
            n=0;
            index++;
            if (index==rgba_arr.height){break;}
            r[index]=[];
            g[index]=[];
            b[index]=[];
            alpha[index]=[];
        }   
    }
    return {
    r: r,
    g: g,
    b: b,
    alpha: alpha
    };
}

In [4]:
// Convert RGBA to 2D Obj
var guitarRGBAObj2D=RGBAtoRGBAObj2D(rawImageData);

In [5]:
// Define an Edge Detection Kernel
var kernel=[[-1, -1, -1],
            [-1,  8, -1],
            [-1, -1, -1]];

In [6]:
// Convolve R,G,B Arrays and reshape Alpha
var convResR=conv.Convolution2D(guitarRGBAObj2D.r,kernel);
var convResG=conv.Convolution2D(guitarRGBAObj2D.g,kernel);
var convResB=conv.Convolution2D(guitarRGBAObj2D.b,kernel);

In [7]:
// Normalization Function
function normalize2D (arr2D) { 

    //Flatten 2D Array
    var flatten=[];
    for (var h=0;h<arr2D.length;h++)
    {
        for(var w=0;w<arr2D[0].length;w++)
        {
            flatten.push(arr2D[h][w]);
            
        }
    }
    
    //Normalize
    var minMax = flatten.reduce((acc, value) => {
      if (value < acc.min) {
         acc.min = value;
      };

      if (value > acc.max) {
         acc.max = value;
      };

      return acc;
   }, {min: Number.POSITIVE_INFINITY, max: Number.NEGATIVE_INFINITY});
   
   var norm_flatten=flatten.map(value => {
      // Account for division by zero
      if (minMax.max === minMax.min) {
         return 1 / flatten.length;
      };

      var diff = minMax.max - minMax.min;
      return Math.round(255*(value - minMax.min) / diff);
   });
   
   
    
    // Back to 2D
    var result=[];
    index=0;
    n=0;
    result[0]=[];
    for(var i = 0; i < flatten.length; i++)
    {
        result[index][n] = norm_flatten[i];
        n++;
        if(n%arr2D[0].length==0 && n!=0)
            {
                n=0;
                index++;
                if (index==arr2D.length){break;}
                result[index]=[];
            }
    } 
    return result;
};

In [8]:
// Normalize Arrays
var normconvResR=normalize2D(convResR);
var normconvResG=normalize2D(convResG);
var normconvResB=normalize2D(convResB);

In [9]:
// Function to Convert 2D Arrays backto RGBA format
function arr2DtoRGBA(r,g,b,alpha_value)
{
    var rgba_arr = [];
    
    for (var h=0;h<r.length;h++)
    {
        for(var w=0;w<r[0].length;w++)
        {
            rgba_arr.push(r[h][w]);
            rgba_arr.push(g[h][w]);
            rgba_arr.push(b[h][w]);
            rgba_arr.push(alpha_value);
        }
    }
    return rgba_arr;
};

[]

In [10]:
// Convert 2D Arrays Back to RGBA
var guitar_edgeRGBA = arr2DtoRGBA(normconvResR, normconvResG, normconvResB, 255);

In [11]:
// Save Array to JSON
var data = JSON.stringify(guitar_edgeRGBA);  
fs.writeFileSync('guitarRGBA.json', data); 

In [12]:
import json
with open('guitarRGBA.json') as jsonfile:
    data = json.dumps(json.load(jsonfile))

In [13]:
%%html
Edge Detection <br>
<canvas id="canvas" width="300" height="300"></canvas>
<canvas id="original"></canvas>

Original Image<br>
<img src="./img/guitar.jpg" align="left" /> 

In [14]:
from IPython.display import Javascript

In [15]:
script="""
  var canvas = document.getElementById('canvas');
  var ctx = canvas.getContext('2d');
  var imageData = ctx.createImageData(258, 258);
  var data = %s+'';
  var dataInts = data.split(",").map(function(x){return parseInt(x)}); 
  var colorR=0;
  var colorG=0;
  var colorB=0;
  
    for (var i = 0; i < dataInts.length; i +=4) {
    if(dataInts[i]<128)
    {
        colorR=0;
    }
    else{colorR=255;};
    
    if(dataInts[i+1]<128)
    {
        colorG=0;
    }
    else{colorG=255;};
    
    if(dataInts[i+2]<128)
    {
        colorB=0;
    }
    else{colorB=255;}
    
      var avg = (colorR + colorG + colorB) / 3;
      imageData.data[i]     = avg;     // red
      imageData.data[i + 1] = avg; // green
      imageData.data[i + 2] = avg; // blue
      imageData.data[i+3] = 255;
    
  };

  ctx.putImageData(imageData, 0, 0);
  
  """%(data)

In [16]:
Javascript(script)

<IPython.core.display.Javascript object>